In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## pytubefix 패키지 설치

In [ ]:
!pip install pytubefix

## 라이브러리 임포트

In [ ]:
import pytubefix
from pytubefix.cli import on_progress
import subprocess
import os
import pandas as pd
import time
from tqdm import tqdm

## 동영상 다운로드 함수

In [ ]:
# 동영상 다운로드 함수
def download_video(youtube_url, output_path):  # 유튜브 url 전체(http~~)와 파일 저장 경로(폴더까지) 입력
    try:
        youtube = pytubefix.YouTube(youtube_url)
        # 원하는 비디오 품질을 선택
        stream = youtube.streams#.all()  # filter(progressive=True)
        # Extract a valid filename from the video URL or title
        #filename = youtube_url.split("=")[-1]+'.mp4' #videoID로 떨어지게
        filename = youtube_url.split("v=")[1].split("&")[0] + ".mp4"
        # 비디오를 원하는 폴더에 다운로드
        stream[0].download(output_path, filename=filename)
        #stream[5].download(output_path, filename=filename)   #오디오 없는 고화질 비디오.
        time.sleep(4)
        print(f"다운로드 완료: {youtube_url}")
    except Exception as e:
        print(f"다운로드 실패: {youtube_url}, 에러: {str(e)}")

## 단일 영상 다운로드

In [ ]:
# video url로부터 지정한 경로(폴더)에 영상 파일 다운로드하기: 단일 유튜브
download_video('https://www.youtube.com/watch?v=GsHTag-UMvY', "/content/drive/MyDrive/Mypython/video/") #폴더 미리 만들지 않아도 실행 가능
#download_video('https://www.youtube.com/watch?v=GsHTag-UMvY', "/content/drive/MyDrive/Mypython/video_1080/") #폴더 미리 만들지 않아도 실행 가능

## 복수 영상 다운로드

In [ ]:
# 복수의 유튜브 분석
# 수집된 유튜브 url 저장한 엑셀 파일 경로 설정
excel_file_path = "/content/drive/MyDrive/Mypython/YouTube_Crawling_중증외상센터.xlsx"

# 엑셀 파일에서 video_id 목록 및 기타 정보 가져오기
video_id_df = pd.read_excel(excel_file_path)
print(video_id_df.columns)

# 모든 동영상에 대해 다운로드
output_folder = "/content/drive/MyDrive/Mypython/video/"  # 절대경로 방식으로 최종 폴더까지 입력. 마지막은 '/'로 끝나야 폴더로 인식.
for url in tqdm(video_id_df['urls'][:10]): #[:10] video_id 10개만 수행, [:] 리스트 전체 수행
    download_video(url, output_folder)

## (참고) 영상스트리밍 가능성 체크

In [ ]:
import pytubefix

youtube_url = 'https://www.youtube.com/watch?v=GsHTag-UMvY'

youtube = pytubefix.YouTube(youtube_url)

# 모든 영상 가져오기
streams = youtube.streams

# 비디오와 오디오가 모두 있는 영상
#streams = youtube.streams.filter(progressive=True)

print("Available streams:")
for i, stream in enumerate(streams):
    print(f"{i}: {stream.resolution} - {stream.mime_type} - {stream.itag} - Audio Codec: {stream.audio_codec}")

# 스트림이 존재하지 않을 경우
if not streams:
    print(f"❌ 스트림을 찾을 수 없습니다: {youtube_url}")
    exit()

# 해상도를 기준으로 숫자 정렬하여 최고 화질 선택
def resolution_to_int(stream):
    return int(stream.resolution[:-1]) if stream.resolution else 0  # "1080p" -> 1080

best_stream = max(streams, key=resolution_to_int)

print(f"\nBest stream selected: {best_stream.resolution} - {best_stream.mime_type} - {best_stream.itag} - Audio Codec: {best_stream.audio_codec}")


## (참고) 오디오 덮어쓰기
- 저화질 영상에 있는 오디오를 추출해서 오디오가 없는 고화질 영상에 추가

## 단일 영상 오디오 덮어쓰기

In [ ]:
import os
import subprocess

# 원본 동영상 경로 (오디오 포함)
input_video = "/content/drive/MyDrive/Mypython/video/GsHTag-UMvY.mp4"
# 대상 동영상 경로 (오디오 추가될 파일)
output_video = "/content/drive/MyDrive/Mypython/video_1080/GsHTag-UMvY.mp4"
# 임시 오디오 파일 경로
audio_file = "/content/drive/MyDrive/Mypython/video/GsHTag-UMvY.wav"

# 오디오 추출
subprocess.run(f'ffmpeg -i "{input_video}" -q:a 0 -map a "{audio_file}" -y', shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)

# 오디오 추가
subprocess.run(f'ffmpeg -i "{output_video}" -i "{audio_file}" -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 -y "{output_video}.temp.mp4"', shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)

# 결과 파일 교체
os.rename(f"{output_video}.temp.mp4", output_video)

# 임시 오디오 파일 삭제
os.remove(audio_file)

print("오디오 추가 완료!")


## 복수 영상(폴더 전체) 오디오 덮어쓰기

In [ ]:
import os
from tqdm import tqdm
import subprocess

# 오디오가 있는 영상들의 폴더
input_folder = '/content/drive/MyDrive/Mypython/video/'
# 오디오를 추가할 영상들의 폴더
output_folder = '/content/drive/MyDrive/Mypython/video_1080/'

def extract_audio(video_path, audio_path):
    """동영상에서 오디오 파일을 추출"""
    command = f"ffmpeg -i \"{video_path}\" -q:a 0 -map a \"{audio_path}\" -y"
    subprocess.run(command, shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)

def add_audio_to_video(video_path, audio_path, output_path):
    """동영상에 오디오 파일 추가"""
    temp_output_path = output_path + ".temp.mp4"
    command = f"ffmpeg -i \"{video_path}\" -i \"{audio_path}\" -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 \"{temp_output_path}\" -y"
    subprocess.run(command, shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    os.rename(temp_output_path, output_path)

def clean_audio_files(folder, extensions=('.mp3', '.wav')):
    """지정된 폴더에서 지정된 확장자의 파일을 삭제"""
    for file_name in os.listdir(folder):
        if file_name.endswith(extensions):
            os.remove(os.path.join(folder, file_name))

def has_audio_track(video_path):
    """동영상 파일에 오디오 트랙이 있는지 확인"""
    command = f"ffmpeg -i \"{video_path}\""
    result = subprocess.run(command, shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    output = result.stderr.decode()
    return "Audio:" in output

# 동영상 파일 리스트 가져오기
video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.mkv', '.avi'))]

# 진행 상황을 표시하기 위해 tqdm 사용
for video_file in tqdm(video_files, desc="Processing videos"):
    try:
        input_video_path = os.path.join(input_folder, video_file)
        output_video_path = os.path.join(output_folder, video_file)
        extracted_audio_path = os.path.join(input_folder, os.path.splitext(video_file)[0] + '.wav')  # 변경된 오디오 형식

        # 출력 파일에 오디오 트랙이 있는지 확인
        if has_audio_track(output_video_path):
            print(f"Skipping {video_file} as it already has an audio track.")
            continue

        # 오디오 추출
        extract_audio(input_video_path, extracted_audio_path)

        # 오디오 추가
        add_audio_to_video(output_video_path, extracted_audio_path, output_video_path)

        # 추출된 오디오 파일 삭제
        if os.path.exists(extracted_audio_path):
            os.remove(extracted_audio_path)
    except Exception as e:
        print(f"Error processing file {video_file}: {e}")

# 입력 폴더에서 남아 있는 mp3 및 wav 파일 삭제
clean_audio_files(input_folder)

print("작업 완료!")


## I-frame 추출 함수

- I-frame(Intra-coded Frame): 비디오 압축에서 독립적으로 디코딩될 수 있는 키 프레임(Key Frame)
- P-frame: 이전 프레임(I/P)와 비교하여 변화된 부분만 저장
- B-frame: 앞뒤 프레임(I/P)을 참조하여 데이터 압축


In [ ]:
def extract_iframes(video_path, base_folder):
    # 비디오 파일 이름에서 확장자를 제외한 기본 이름 가져오기
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    output_folder = os.path.join(base_folder, base_name)

    # 출력 폴더가 존재하지 않으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # ffprobe 명령어를 사용하여 총 프레임 수 가져오기
    try:
        ffprobe_cmd = [
            'ffprobe', '-v', 'error', '-select_streams', 'v:0',
            '-show_entries', 'stream=nb_frames', '-of', 'default=nokey=1:noprint_wrappers=1',
            video_path
        ]
        total_frames = subprocess.check_output(ffprobe_cmd).decode().strip()
        total_frames = int(total_frames) if total_frames.isdigit() else 0  # 'N/A' 방지
    except subprocess.CalledProcessError as e:
        print(f"ffprobe 실행 오류: {e}")
        total_frames = 0  # 오류 발생 시 기본값

    # ffmpeg 명령어 구성
    command = [
        'ffmpeg', '-i', video_path, '-vf', 'select=eq(pict_type\\,I)',
        '-vsync', 'vfr', '-f', 'image2', f'{output_folder}/{base_name}_%d.jpg'
    ]

    try:
        subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    except subprocess.CalledProcessError as e:
        print(f"ffmpeg 실행 오류: {e}")
        return total_frames, 0  # 오류 발생 시 키프레임 수 0 반환

    # 추출된 I-프레임(키프레임) 수 계산
    i_frame_count = len([filename for filename in os.listdir(output_folder) if filename.endswith('.jpg')])

    return total_frames, i_frame_count

## 단일 영상 I-frame 추출

In [ ]:
# 영상파일 1개 실행
video_path = '/content/drive/MyDrive/Mypython/video/GsHTag-UMvY.mp4'
base_folder = '/content/drive/MyDrive/Mypython/i_frame/'

total_frames, total_keyframes = extract_iframes(video_path=video_path, base_folder=base_folder)

print("총 프레임 수:", total_frames)
print("총 키프레임 수:", total_keyframes)

## 복수 영상 I-frame 추출

In [ ]:
# 특정 폴더 내의 모든 영상 파일 경로 수집
def get_all_video_paths(folder_path, video_extensions=('.mp4', '.avi', '.mov', '.mkv')):
    video_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(video_extensions):
                video_paths.append(os.path.join(root, file))
    return video_paths

# 폴더 경로 및 기본 폴더 설정
folder_path = '/content/drive/MyDrive/Mypython/video/'
base_folder = '/content/drive/MyDrive/Mypython/i_frame/'

# 특정 폴더 내의 모든 영상 파일 경로 가져오기
video_paths = get_all_video_paths(folder_path)

# 영상파일별 total_frames, total_keyframes 저장
result = []
for video_path in tqdm(video_paths, desc="Processing videos"):
    try:
        total_frames, total_keyframes = extract_iframes(video_path, base_folder)
        result.append([video_path, total_frames, total_keyframes])
    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        result.append([video_path, '', ''])

# 결과 출력
for item in result:
    print(f"Video: {item[0]}, Total Frames: {item[1]}, Total Keyframes: {item[2]}")


## (참고) I-frame 추출 결과를 엑셀로 저장

In [ ]:
import pandas as pd
import os

# 파일명에서 확장자 제거 (mp4 파일만 해당)
formatted_result = [[os.path.splitext(os.path.basename(item[0]))[0] if item[0].endswith('.mp4') else item[0], item[1], item[2]] for item in result]

# 데이터프레임 생성
df = pd.DataFrame(formatted_result, columns=['Video File', 'Total Frames', 'Total Keyframes'])

# 엑셀 파일 저장 경로
excel_path = '/content/drive/MyDrive/Mypython/i_frame/iframe_result.xlsx'

# 엑셀 파일 저장
df.to_excel(excel_path, index=False)

print(f"엑셀 파일 저장 완료: {excel_path}")
